# HW1: Frame-Level Speech Recognition

In this homework, you will be working with MFCC data consisting of 15 features at each time step/frame. Your model should be able to recognize the phoneme occured in that frame.

# Libraries

In [ ]:
!pip install torchsummaryX wandb --quiet
!pip install tqdm

     |████████████████████████████████| 1.8 MB 5.2 MB/s 
     |████████████████████████████████| 162 kB 45.7 MB/s 
     |████████████████████████████████| 181 kB 55.3 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 158 kB 50.0 MB/s 
     |████████████████████████████████| 157 kB 37.0 MB/s 
     |████████████████████████████████| 157 kB 49.2 MB/s 
     |████████████████████████████████| 157 kB 46.2 MB/s 
     |████████████████████████████████| 157 kB 38.1 MB/s 
     |████████████████████████████████| 157 kB 42.5 MB/s 
     |████████████████████████████████| 157 kB 44.6 MB/s 
     |████████████████████████████████| 157 kB 38.1 MB/s 
     |████████████████████████████████| 156 kB 43.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#!pip install torch==1.2.0 This installs adamw but cannot do GELU afterward

In [ ]:
import tqdm
import torch
print(torch.__version__)
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import accuracy_score
import gc
import zipfile
import pandas as pd
from tqdm.auto import tqdm
import os
import datetime
import wandb
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

1.12.1+cu113
Device:  cuda


In [ ]:
### If you are using colab, you can import google drive to save model checkpoints in a folder
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
### PHONEME LIST
PHONEMES = [
            'SIL',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '<sos>', '<eos>']

# Kaggle

This section contains code that helps you install kaggle's API, creating kaggle.json with you username and API key details. Make sure to input those in the given code to ensure you can download data from the competition successfully.

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"joonghochoi","key":"7420d9855539804c3a24a55d7add20dc"}') 
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59 kB 3.1 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73275 sha256=437ac220d5a7bf1a0d8d30dcab44802f7d47b75c9644435ff8897cd3b8f329bc
  Stored in directory: /root/.cache/pip/wheels/de/f7/d8/c3902cacb7e62cb611b1ad343d7cc07f42f7eb76ae3a52f3d1
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [ ]:
# commands to download data from kaggle
#Temporary. Data disappears. Need to redownload every time.

!kaggle competitions download -c 11-785-f22-hw1p2
!mkdir '/content/data'

!unzip -qo '11-785-f22-hw1p2.zip' -d '/content/data'

 99% 2.11G/2.13G [00:09<00:00, 234MB/s]
100% 2.13G/2.13G [00:10<00:00, 228MB/s]


In [ ]:

# import numpy as np
# res=0
# c=0
# for item in transcripts:
#     unique, counts = np.unique(item, return_counts=True)
#     c+=counts[np.where(unique=="SIL")][0]
# print(c)

In [ ]:
# import numpy as np
# res=0
# d=dict()
# for item in transcripts:
#     unique, counts = np.unique(item, return_counts=True)
#     length=len(unique)
#     for i in range(length):
#       item=unique[i]
#       if item not in d:
#         d[item]=0
#       else:
#         d[item]+=counts[i]
# print(d)

# Dataset

This section covers the dataset/dataloader class for speech data. You will have to spend time writing code to create this class successfully. We have given you a lot of comments guiding you on what code to write at each stage, from top to bottom of the class. Please try and take your time figuring this out, as it will immensely help in creating dataset/dataloader classes for future homeworks.

Before running the following cells, please take some time to analyse the structure of data. Try loading a single MFCC and its transcipt, print out the shapes and print out the values. Do the transcripts look like phonemes?

In [ ]:
# import os
# import numpy as np
# a_names=sorted(os.listdir("/content/data/train-clean-100/mfcc"))
# mfccs=[]
# for i in range(0,len(a_names)):
#   mfcc = np.load("/content/data/train-clean-100/mfcc/"+a_names[i]) 
#   mfccs.append(mfcc)
# print(len(mfccs))
# mfccs1=np.concatenate(mfccs)
# mfccs1
# mfccs2=np.pad(mfccs1, ((1, 1), (0, 0)), 'constant', constant_values=0)
# mfccs2
# print(mfccs1.shape)
# print(mfccs2.shape)
# print(mfccs2[5-2:5+2+1].shape)
#x=np.array(['0','1'])
#x
#x.astype(np.int)

In [ ]:
# Dataset class to load train and validation data

class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, data_path, context, offset=0, partition= "train", limit=-1): # Feel free to add more arguments

        self.context = context
        self.offset = offset
        self.data_path = data_path
        if partition=="train":
          self.mfcc_dir = data_path+partition+"-clean-100/mfcc" # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data
          self.transcript_dir = data_path+partition+"-clean-100/transcript" # TODO: Transcripts directory - use partition to acces train/dev directories from kaggle data
        if partition=="dev":
          self.mfcc_dir = data_path+partition+"-clean/mfcc" # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data
          self.transcript_dir = data_path+partition+"-clean/transcript" 

        mfcc_names = sorted(os.listdir(self.mfcc_dir)) # TODO: List files in X_dir using os.listdir in sorted order, optionally subset using limit to slice the number of files you load
        transcript_names = sorted(os.listdir(self.transcript_dir))# TODO: List files in Y_dir using os.listdir in sorted order, optionally subset using limit to slice the number of files you load

        assert len(mfcc_names) == len(transcript_names) # Making sure that we have the same no. of mfcc and transcripts

        self.mfccs, self.transcripts = [], []

        # TODO:
        # Iterate through mfccs and transcripts
        for i in range(0, len(mfcc_names)):
        #   Load a single mfcc
            if partition=="train":
              mfcc = np.load(data_path+partition+"-clean-100/mfcc/"+mfcc_names[i])
              
        #   Optionally do Cepstral Normalization of mfcc. Ans: Just normal 
              mfcc=((mfcc-np.mean(mfcc,axis=0))/np.std(mfcc,axis=0))
        #   Load the corresponding transcript
              transcript = np.load(data_path+partition+"-clean-100/transcript/"+transcript_names[i]) 
            else:
              mfcc = np.load(data_path+partition+"-clean/mfcc/"+mfcc_names[i])
              mfcc=((mfcc-np.mean(mfcc,axis=0))/np.std(mfcc,axis=0))

              transcript = np.load(data_path+partition+"-clean/transcript/"+transcript_names[i]) 

            # Remove [SOS] and [EOS] from the transcript (Is there an efficient way to do this 
            # without traversing through the transcript? Converting to list. But that'd be ineffective)
            transcript=transcript[1:-1]
            
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)

        

        # NOTE:
        # Each mfcc is of shape T1 x 15, T2 x 15, ...
        # Each transcript is of shape (T1+2) x 15, (T2+2) x 15 before removing [SOS] and [EOS]

        # TODO: Concatenate all mfccs in self.X such that the final shape is T x 15 (Where T = T1 + T2 + ...) 
        self.mfccs = np.concatenate(self.mfccs,axis=0) #no need to specify axis 

        # TODO: Concatenate all transcripts in self.Y such that the final shape is (T,) meaning, each time step has one phoneme output
        #ex. first one only has all SIL. 
        self.transcripts = np.concatenate(self.transcripts,axis=0) #no need to specify axis
        # Hint: Use numpy to concatenate

        # Take some time to think about what we have done. self.mfcc is an array of the format (Frames x Features). Our goal is to recognize phonemes of each frame
        # From hw0, you will be knowing what context is. We can introduce context by padding zeros on top and bottom of self.mfcc
        
        #self.mfccs = np.pad(self.mfccs, ((1, 1), (0, 0)), 'constant', constant_values=0) # TODO.   #hw0 ex6. 
        self.mfccs = np.pad(self.mfccs, ((self.context, self.context), (0, 0)), 'constant', constant_values=0) # TODO.   #hw0 ex6. 

        #masking = torchaudio.transforms.TimeMasking(time_mask_param=80) #apply to each mfcc or just do it once in getitem 


        #ex.context=3. 3 zero padding at top and 3 at hbottom 
        

        # These are the available phonemes in the transcript
        self.phonemes = [
            'SIL',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '<sos>', '<eos>']
        # But the neural network cannot predict strings as such. Instead we map these phonemes to integers
        #ex. self.phonemes.index('SIL')  ->0
        
        self.transcripts=np.array([self.phonemes.index(yy) for yy in self.transcripts])


        # Length of the dataset is now the length of concatenated mfccs/transcripts
        #self.length = len(self.mfccs)
        self.length=len(self.transcripts)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        #If index is 0 and context is 1, it shoukd be [0,3]
        #If index is 1 and context 1, it should be [1,4]
        frames=self.mfccs[ind:ind+2*self.context+1]

        #frames = self.masking(frames). If this doesn't work, move it to after flatten. If still doesn't work, try transposing. 




        # After slicing, you get an array of shape 2*context+1 x 15. But our MLP needs 1d data and not 2d.
        frames = frames.flatten() # TODO: Flatten to get 1d data
        #frames= ((frames-np.mean(frames))/np.std(frames))

        frames = torch.FloatTensor(frames) # Convert to tensors
        phoneme = torch.tensor(self.transcripts[ind])       

        return frames, phoneme

In [ ]:
class AudioTestDataset(torch.utils.data.Dataset):
    pass

    # TODO: Create a test dataset class similar to the previous class but you dont have transcripts for this
    # Imp: Read the mfccs in sorted order, do NOT shuffle the data here or in your dataloader.
    def __init__(self, data_path, context, offset=0, partition= "test", limit=-1): # Feel free to add more arguments

        self.context = context
        self.offset = offset
        self.data_path = data_path

        self.mfcc_dir = data_path+partition+"-clean/mfcc" # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data

        mfcc_names = sorted(os.listdir(self.mfcc_dir)) # TODO: List files in X_dir using os.listdir in sorted order, optionally subset using limit to slice the number of files you load

        self.mfccs= []

        # TODO:
        # Iterate through mfccs and transcripts
        for i in range(0, len(mfcc_names)):
        #   Load a single mfcc
            mfcc = np.load(data_path+partition+"-clean/mfcc/"+mfcc_names[i])
        #   Optionally do Cepstral Normalization of mfcc. Ans: Just normal 
            mfcc=((mfcc-np.mean(mfcc,axis=0))/np.std(mfcc,axis=0))
        
      
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)

        

        # NOTE:
        # Each mfcc is of shape T1 x 15, T2 x 15, ...
        # Each transcript is of shape (T1+2) x 15, (T2+2) x 15 before removing [SOS] and [EOS]

        # TODO: Concatenate all mfccs in self.X such that the final shape is T x 15 (Where T = T1 + T2 + ...) 
        self.mfccs = np.concatenate(self.mfccs,axis=0) #no need to specify axis 

       # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.mfccs)

        # Take some time to think about what we have done. self.mfcc is an array of the format (Frames x Features). Our goal is to recognize phonemes of each frame
        # From hw0, you will be knowing what context is. We can introduce context by padding zeros on top and bottom of self.mfcc
        #self.mfccs = np.pad(self.mfccs, ((1, 1), (0, 0)), 'constant', constant_values=0) # TODO.   #hw0 ex6. 
        self.mfccs = np.pad(self.mfccs, ((self.context, self.context), (0, 0)), 'constant', constant_values=0)

        
        

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        frames=self.mfccs[ind:ind+2*self.context+1]
        #frames = self.mfccs[ind-self.context,ind+self.context+1] # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        # After slicing, you get an array of shape 2*context+1 x 15. But our MLP needs 1d data and not 2d.
        frames = frames.flatten() # TODO: Flatten to get 1d data

        frames = torch.FloatTensor(frames) # Convert to tensors
        #phoneme = torch.tensor(self.transcripts[ind])       

        return frames #,phoneme

# Parameters Configuration

Storing your parameters and hyperparameters in a single configuration dictionary makes it easier to keep track of them during each experiment. It can also be used with weights and biases to log your parameters for each experiment and keep track of them across multiple experiments. 

In [ ]:
config = {
    'epochs': 8, #15
    'batch_size' : 4096, #original 1024
    'context' : 25,  #Reached 27->input_size=825->83.89%
    'learning_rate' : 1e-3,
    'architecture' : 'very-low-cutoff',
    'weight_decay' :1e-4, #tried 1e-2. try 1e-6 after if this doesn't work
    'reLU': 1,
    
    # Add more as you need them - e.g dropout values, weight decay, scheduler parameters
}

#Moreover, it will take more time to run many small steps. On the opposite, big batch size can really speed up your training, and even have better generalization performances.
#Using too large a batch size can have a negative effect on the accuracy of your network during training since it reduces the stochasticity of the gradient descent. 
#With bigger batches (and therefore fewer per epoch) you will have fewer gradient updates per epoch.

In [ ]:
# for i in train_data:
#   x, y = i
#   print(x)
#   print(len(x))
#   print(y)
#   break

# Create Datasets

In [ ]:
train_data = AudioDataset( data_path="/content/data/", context=config['context'],offset=0,partition='train',limit= -1 ) #TODO: Create a dataset object using the AudioDataset class for the training data 
                                                   
val_data = AudioDataset( data_path="/content/data/", context=config['context'],offset=0,partition='dev',limit= -1 ) # TODO: Create a dataset object using the AudioDataset class for the validation data 

test_data = AudioTestDataset( data_path="/content/data/",context=config['context'],offset=0,partition='test',limit= -1 ) # TODO: Create a dataset object using the AudioTestDataset class for the test data 

In [ ]:
# Define dataloaders for train, val and test datasets
# Dataloaders will yield a batch of frames and phonemes of given batch_size at every iteration
train_loader = torch.utils.data.DataLoader(train_data, num_workers= 4,
                                           batch_size=config['batch_size'], pin_memory= True,
                                           shuffle= True)

val_loader = torch.utils.data.DataLoader(val_data, num_workers= 2,
                                         batch_size=config['batch_size'], pin_memory= True,
                                         shuffle= False)

test_loader = torch.utils.data.DataLoader(test_data, num_workers= 2, 
                                          batch_size=config['batch_size'], pin_memory= True, 
                                          shuffle= False)


print("Batch size: ", config['batch_size'])
print("Context: ", config['context'])
print("Input size: ", (2*config['context']+1)*15)
print("Output symbols: ", len(PHONEMES))

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  4096
Context:  25
Input size:  765
Output symbols:  42
Train dataset samples = 36191134, batches = 8836
Validation dataset samples = 1937496, batches = 474
Test dataset samples = 1943253, batches = 475


In [ ]:
# Testing code to check if your data loaders are working
for i, data in enumerate(train_loader):
    frames, phoneme = data
    print(frames.shape, phoneme.shape)
    print(phoneme)
    break

torch.Size([4096, 765]) torch.Size([4096])
tensor([31, 10, 17,  ..., 14,  9, 31])


# Network Architecture


This section defines your network architecture for the homework. We have given you a sample architecture that can easily clear the very low cutoff for the early submission deadline.

In [ ]:
# This architecture will make you cross the very low cutoff
# However, you need to run a lot of experiments to cross the medium or high cutoff
class Network(torch.nn.Module):

    def __init__(self, context):

        super(Network, self).__init__()

        input_size = (2*context + 1) * 15 #Why is this the case?
        output_size = 40 #Why?

        self.model=torch.nn.Sequential(
            torch.nn.Linear(in_features=input_size,out_features=2048,bias=True),
            # torch.nn.BatchNorm1d(2048,eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            torch.nn.BatchNorm1d(2048,eps=1e-05, affine=True, track_running_stats=True),
            torch.nn.GELU(),
            # torch.nn.Dropout(p=0.1,inplace=False),
            torch.nn.Dropout(p=0.2,inplace=False),

            torch.nn.Linear(in_features=2048,out_features=2048,bias=True),
            #torch.nn.BatchNorm1d(2048,eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            torch.nn.BatchNorm1d(2048,eps=1e-05, affine=True, track_running_stats=True),
            torch.nn.GELU(approximate='none'),
            # torch.nn.Dropout(p=0.25,inplace=False),
            torch.nn.Dropout(p=0.30,inplace=False),

            torch.nn.Linear(in_features=2048,out_features=2048,bias=True),
            # torch.nn.BatchNorm1d(2048,eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            torch.nn.BatchNorm1d(2048,eps=1e-05, affine=True, track_running_stats=True),
            torch.nn.GELU(approximate='none'),
            # torch.nn.Dropout(p=0.25,inplace=False),
            torch.nn.Dropout(p=0.30,inplace=False),

            torch.nn.Linear(in_features=2048,out_features=2048,bias=True),
            # torch.nn.BatchNorm1d(2048,eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            torch.nn.BatchNorm1d(2048,eps=1e-05, affine=True, track_running_stats=True),
            torch.nn.GELU(approximate='none'),
            # torch.nn.Dropout(p=0.25,inplace=False),
            torch.nn.Dropout(p=0.30,inplace=False),

            torch.nn.Linear(in_features=2048,out_features=1024,bias=True),
            # torch.nn.BatchNorm1d(1024,eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            torch.nn.BatchNorm1d(1024,eps=1e-05,  affine=True, track_running_stats=True),
            torch.nn.GELU(approximate='none'),
            # torch.nn.Dropout(p=0.25,inplace=False),
            torch.nn.Dropout(p=0.30,inplace=False),

            torch.nn.Linear(in_features=1024,out_features=1024,bias=True),
            # torch.nn.BatchNorm1d(1024,eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            torch.nn.BatchNorm1d(1024,eps=1e-05, affine=True, track_running_stats=True),
            torch.nn.GELU(approximate='none'),
            # torch.nn.Dropout(p=0.25,inplace=False),
            torch.nn.Dropout(p=0.30,inplace=False),

            torch.nn.Linear(in_features=1024,out_features=1024,bias=True),
            # torch.nn.BatchNorm1d(1024,eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            torch.nn.BatchNorm1d(1024,eps=1e-05, affine=True, track_running_stats=True),
            torch.nn.GELU(approximate='none'),
            # torch.nn.Dropout(p=0.25,inplace=False),
            torch.nn.Dropout(p=0.30,inplace=False),

            torch.nn.Linear(in_features=1024,out_features=1024,bias=True),
            # torch.nn.BatchNorm1d(1024,eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            torch.nn.BatchNorm1d(1024,eps=1e-05, affine=True, track_running_stats=True),
            torch.nn.GELU(approximate='none'),
            # torch.nn.Dropout(p=0.25,inplace=False),
            torch.nn.Dropout(p=0.30,inplace=False),

            torch.nn.Linear(in_features=1024,out_features=40,bias=True),
        )

    def forward(self, x):
        out = self.model(x)

        return out



# Define Model, Loss Function and Optimizer

---



Here we define the model, loss function, optimizer and optionally a learning rate scheduler. 

In [ ]:
input_size = 15*(2*config['context'] + 1)
model = Network(config['context']).to(device)
frames,phoneme = next(iter(train_loader))
# Check number of parameters of your network - Remember, you are limited to 20 million parameters for HW1 (including ensembles)
summary(model, frames.to(device))

                         Kernel Shape  Output Shape     Params  Mult-Adds
Layer                                                                    
0_model.Linear_0          [765, 2048]  [4096, 2048]  1.568768M   1.56672M
1_model.BatchNorm1d_1          [2048]  [4096, 2048]     4.096k     2.048k
2_model.GELU_2                      -  [4096, 2048]          -          -
3_model.Dropout_3                   -  [4096, 2048]          -          -
4_model.Linear_4         [2048, 2048]  [4096, 2048]  4.196352M  4.194304M
5_model.BatchNorm1d_5          [2048]  [4096, 2048]     4.096k     2.048k
6_model.GELU_6                      -  [4096, 2048]          -          -
7_model.Dropout_7                   -  [4096, 2048]          -          -
8_model.Linear_8         [2048, 2048]  [4096, 2048]  4.196352M  4.194304M
9_model.BatchNorm1d_9          [2048]  [4096, 2048]     4.096k     2.048k
10_model.GELU_10                    -  [4096, 2048]          -          -
11_model.Dropout_11                 - 

/usr/local/lib/python3.7/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


Kernel Shape  Output Shape     Params  Mult-Adds
Layer                                                                    
0_model.Linear_0          [765, 2048]  [4096, 2048]  1568768.0  1566720.0
1_model.BatchNorm1d_1          [2048]  [4096, 2048]     4096.0     2048.0
2_model.GELU_2                      -  [4096, 2048]        NaN        NaN
3_model.Dropout_3                   -  [4096, 2048]        NaN        NaN
4_model.Linear_4         [2048, 2048]  [4096, 2048]  4196352.0  4194304.0
5_model.BatchNorm1d_5          [2048]  [4096, 2048]     4096.0     2048.0
6_model.GELU_6                      -  [4096, 2048]        NaN        NaN
7_model.Dropout_7                   -  [4096, 2048]        NaN        NaN
8_model.Linear_8         [2048, 2048]  [4096, 2048]  4196352.0  4194304.0
9_model.BatchNorm1d_9          [2048]  [4096, 2048]     4096.0     2048.0
10_model.GELU_10                    -  [4096, 2048]        NaN        NaN
11_model.Dropout_11                 -  [4096, 2048]        NaN        NaN
12_model.Linear_12       [2048, 2048]  [4096, 2048]  4196352.0  4194304.0
13_model.BatchNorm1d_13        [2048]  [4096, 2048]     4096.0     2048.0
14_model.GELU_14                    -  [4096, 2048]        NaN        NaN
15_model.Dropout_15                 -  [4096, 2048]        NaN        NaN
16_model.Linear_16       [2048, 1024]  [4096, 1024]  2098176.0  2097152.0
17_model.BatchNorm1d_17        [1024]  [4096, 1024]     2048.0     1024.0
18_model.GELU_18                    -  [4096, 1024]        NaN        NaN
19_model.Dropout_19                 -  [4096, 1024]        NaN        NaN
20_model.Linear_20       [1024, 1024]  [4096, 1024]  1049600.0  1048576.0
21_model.BatchNorm1d_21        [1024]  [4096, 1024]     2048.0     1024.0
22_model.GELU_22                    -  [4096, 1024]        NaN        NaN
23_model.Dropout_23                 -  [4096, 1024]        NaN        NaN
24_model.Linear_24       [1024, 1024]  [4096, 1024]  1049600.0  1048576.0
25_model.BatchNorm1d_25        [1024]  [4096, 1024]     2048.0     1024.0
26_model.GELU_26                    -  [4096, 1024]        NaN        NaN
27_model.Dropout_27                 -  [4096, 1024]        NaN        NaN
28_model.Linear_28       [1024, 1024]  [4096, 1024]  1049600.0  1048576.0
29_model.BatchNorm1d_29        [1024]  [4096, 1024]     2048.0     1024.0
30_model.GELU_30                    -  [4096, 1024]        NaN        NaN
31_model.Dropout_31                 -  [4096, 1024]        NaN        NaN
32_model.Linear_32         [1024, 40]    [4096, 40]    41000.0    40960.0

In [ ]:
#criterion = nn.CrossEntropyLoss(label_smoothing=0.2) #Defining Loss function 
criterion = nn.CrossEntropyLoss(label_smoothing=0.3)
#label smoothing=difference between labels gets smoothed. A way to prevent overfitting. Other loss functions also use this 

#optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate']) #Defining Optimizer
# optimizer = torch.optim.AdamW(model.parameters(), lr=config['learning_rate'])
optimizer = torch.optim.AdamW(model.parameters(), lr=config['learning_rate'],weight_decay=0.01)



#Even if scheduler, defining learning rate is important, as it sets the starting point. Scheduler will update accordingly but never go below min_lr
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
#                                                                mode='max',  #max for maximizing validation accuracy 
#                                                                factor=0.2,
#                                                                patience=2,
#                                                                min_lr=1e-6,
#                                                                verbose=True)

scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones=[3,5,7],gamma=0.5) 
#15th,22th epoch to affect the learning. 1e-3 can do well until around 15th. 
#gamma: Multiplicative factor of learning rate decay


# Recommended : Define Scheduler for Learning Rate, including but not limited to StepLR, MultiStepLR, CosineAnnealingLR, ReduceLROnPlateau, etc. 
# You can refer to Pytorch documentation for more information on how to use them.

#StepLR: different scheduler. https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.StepLR.html

# Training and Validation Functions

This section covers the training, and validation functions for each epoch of running your experiment with a given model architecture. The code has been provided to you, but we recommend going through the comments to understand the workflow to enable you to write these loops for future HWs.

In [ ]:
torch.cuda.empty_cache()
gc.collect()

144

In [ ]:
def train(model, optimizer, criterion, dataloader):

    model.train()
    train_loss = 0.0 #Monitoring Loss
    
    batch_bar=tqdm(total=len(train_loader),dynamic_ncols=True,leave=False,position=0,desc='Train')
    
    for iter, (mfccs, phonemes) in enumerate(dataloader):

        ### Move Data to Device (Ideally GPU)
        mfccs = mfccs.to(device)
        phonemes = phonemes.to(device)

        ### Forward Propagation
        logits = model(mfccs)

        ### Loss Calculation
        loss = criterion(logits, phonemes)

        # ###L2 regularization 
        # l2_lambda = 0.001
        # l2_norm = sum(p.pow(2.0).sum()
        #                for p in model.parameters())
        # train_loss = loss.item() + l2_lambda * l2_norm      leads to training error to be zero
        
        train_loss += loss.item()

        ### Initialize Gradients
        optimizer.zero_grad()

        ### Backward Propagation
        loss.backward()

        ### Gradient Descent
        optimizer.step() #update parameter based on gradient
        #scheduler.step(loss)
        #scheduler.step(). #monitor parameter and update lr 
        batch_bar.update()
    batch_bar.close()
    train_loss /= len(dataloader)
    #scheduler.step(train_loss)
    return train_loss

In [ ]:
def eval(model, dataloader):

    model.eval() # set model in evaluation mode

    phone_true_list = []
    phone_pred_list = []

    for i, data in enumerate(dataloader):

        frames, phonemes = data
        ### Move data to device (ideally GPU)
        frames, phonemes = frames.to(device), phonemes.to(device) 

        with torch.inference_mode(): # makes sure that there are no gradients computed as we are not training the model now
            ### Forward Propagation
            logits = model(frames)

        ### Get Predictions
        predicted_phonemes = torch.argmax(logits, dim=1)
        
        ### Store Pred and True Labels
        phone_pred_list.extend(predicted_phonemes.cpu().tolist())
        phone_true_list.extend(phonemes.cpu().tolist())
        
        # Do you think we need loss.backward() and optimizer.step() here?
    
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    ### Calculate Accuracy
    accuracy = sklearn.metrics.accuracy_score(phone_pred_list, phone_true_list) 
    return accuracy*100

# Weights and Biases Setup

This section is to enable logging metrics and files with Weights and Biases. Please refer to wandb documentationa and recitation 0 that covers the use of weights and biases for logging, hyperparameter tuning and monitoring your runs for your homeworks. Using this tool makes it very easy to show results when submitting your code and models for homeworks, and also extremely useful for study groups to organize and run ablations under a single team in wandb. 

We have written code for you to make use of it out of the box, so that you start using wandb for all your HWs from the beginning.

In [ ]:
wandb.login(key="b60bfe3e01a4340c8db67c2772c12aaf0441572f") #API Key is in your wandb account, under settings (wandb.ai/settings)
#recitation 0 https://colab.research.google.com/drive/1gVwGcSThCzIJ9_k5HYEbpdp9QOtI2Y_I?usp=sharing
#API key here: https://wandb.ai/authorize

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# Create your wandb run
run = wandb.init(
    name = "Good_attempt7.3: manual scheduler", ### Wandb creates random run names if you skip this field, we recommend you give useful names
    reinit=True, ### Allows reinitalizing runs when you re-run this cell
    project="hw1p2", ### Project should be created in your wandb account 
    config=config ### Wandb Config for your run
)

wandb: Currently logged in as: jc78 (idl-fall22). Use `wandb login --relogin` to force relogin


In [ ]:
### Save your model architecture as a string with str(model) 
model_arch = str(model)

### Save it in a txt file 
arch_file = open("model_arch.txt", "w")
file_write = arch_file.write(model_arch)
arch_file.close()

### log it in your wandb run with wandb.save()
wandb.save('model_arch.txt')

['/content/wandb/run-20220928_202759-3n9hw842/files/model_arch.txt',
 '/content/wandb/run-20220928_202759-3n9hw842/files/model_arch.txt']

# Experiment

Now, it is time to finally run your ablations! Have fun!

In [ ]:
PATH="./drive/MyDrive/2022_Fall/11485model.pth"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
print(checkpoint['acc'])

87.26521241850305


In [ ]:
#optimizer.param_groups[0]['lr'] = 0.0005 #0.0005

In [ ]:
print(optimizer)

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    eps: 1e-08
    foreach: None
    initial_lr: 0.001
    lr: 0.0005
    maximize: False
    weight_decay: 0.01
)


In [ ]:
# Iterate over 5epochs to train and evaluate your loaded model
import sklearn
torch.cuda.empty_cache()

best_acc = 0.0 ### Monitor best accuracy in your run

for epoch in range(8):
    print("\nEpoch {}/{}".format(epoch+1, 8)) #

    train_loss = train(model, optimizer, criterion, train_loader)
    #scheduler.step(train_loss) Validation Accuracy: 1.71% without anything in (). with train_loss in (), slightly better but still 5.92%
    print('Epoch {}, lr {}'.format(epoch, optimizer.param_groups[0]['lr']))
    accuracy = eval(model, val_loader)
    
    print("\tTrain Loss: {:.4f}".format(train_loss))
    print("\tValidation Accuracy: {:.2f}%".format(accuracy))


    ### Log metrics at each epoch in your run - Optionally, you can log at each batch inside train/eval functions (explore wandb documentation/wandb recitation)
    wandb.log({"train loss": train_loss, "validation accuracy": accuracy})
    print(optimizer.param_groups[0]['lr'])
    ### Save checkpoint if accuracy is better than your current best
    if accuracy >= best_acc:

      ### Save checkpoint with information you want
      torch.save({'epoch': epoch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss': train_loss,
              'acc': accuracy}, 
        './model_checkpoint.pth')
       
      ### Save checkpoint in wandb
      wandb.save('checkpoint.pth')


    # Is your training time very high? Look into mixed precision training if your GPU (Tesla T4, V100, etc) can make use of it 
    # Refer - https://pytorch.org/docs/stable/notes/amp_examples.html

### Finish your wandb run
run.finish()


Epoch 1/8


Train:   0%|          | 0/8836 [00:00<?, ?it/s]

Epoch 0, lr 0.0005
	Train Loss: 1.8849
	Validation Accuracy: 86.93%
0.0005

Epoch 2/8


Train:   0%|          | 0/8836 [00:00<?, ?it/s]

Epoch 1, lr 0.0005
	Train Loss: 1.8868
	Validation Accuracy: 86.93%
0.0005

Epoch 3/8


...

Epoch 5, lr 0.0005
	Train Loss: 1.8883
	Validation Accuracy: 86.99%
0.0005

Epoch 7/8


Train:   0%|          | 0/8836 [00:00<?, ?it/s]

Epoch 6, lr 0.0005
	Train Loss: 1.8884
	Validation Accuracy: 86.99%
0.0005

Epoch 8/8


Train:   0%|          | 0/8836 [00:00<?, ?it/s]

Epoch 7, lr 0.0005
	Train Loss: 1.8885
	Validation Accuracy: 87.01%
0.0005


In [ ]:
torch.save({'epoch': epoch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss': train_loss,
              'acc': accuracy}, 
        './drive/MyDrive/2022_Fall/11485model.pth')

In [ ]:
PATH="./drive/MyDrive/2022_Fall/11485model.pth"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
print(checkpoint['acc'])

87.26438661034655


In [ ]:
#Uncomment for the first time training. Iterate over number of epochs to train and evaluate your model
# import sklearn
# torch.cuda.empty_cache()

# best_acc = 0.0 ### Monitor best accuracy in your run

# for epoch in range(config['epochs']):
#     print("\nEpoch {}/{}".format(epoch+1, config['epochs'])) #

#     train_loss = train(model, optimizer, criterion, train_loader)
#     #scheduler.step(train_loss) V. alidation Accuracy: 1.71% without anything in (). with train_loss in (), slightly better but still 5.92%
#     print('Epoch {}, lr {}'.format(epoch, optimizer.param_groups[0]['lr']))
#     accuracy = eval(model, val_loader)
    
#     print("\tTrain Loss: {:.4f}".format(train_loss))
#     print("\tValidation Accuracy: {:.2f}%".format(accuracy))


#     ### Log metrics at each epoch in your run - Optionally, you can log at each batch inside train/eval functions (explore wandb documentation/wandb recitation)
#     wandb.log({"train loss": train_loss, "validation accuracy": accuracy})
#     print(optimizer.param_groups[0]['lr'])
#     ### Save checkpoint if accuracy is better than your current best
#     if accuracy >= best_acc:

#       ### Save checkpoint with information you want
#       torch.save({'epoch': epoch,
#               'model_state_dict': model.state_dict(),
#               'optimizer_state_dict': optimizer.state_dict(),
#               'loss': train_loss,
#               'acc': accuracy}, 
#         './model_checkpoint.pth')
       
#       ### Save checkpoint in wandb
#       wandb.save('checkpoint.pth')


#     # Is your training time very high? Look into mixed precision training if your GPU (Tesla T4, V100, etc) can make use of it 
#     # Refer - https://pytorch.org/docs/stable/notes/amp_examples.html

# ### Finish your wandb run
# run.finish()

# Testing and submission to Kaggle

Before we get to the following code, make sure to see the format of submission given in *random_submission.csv*. Once you have done so, it is time to fill the following function to complete your inference on test data. Refer the eval function from previous cells to get an idea of how to go about completing this function.

In [ ]:
#similar to evaluate function
def test(model, test_loader):
  ### What you call for model to perform inference?
  model.eval()

  ### List to store predicted phonemes of test data
  test_predictions = []

  ### Which mode do you need to avoid gradients?

  #inference mode also works.      torch.inference_mode  
  #torch.no_grad AttributeError: __enter__
  with torch.no_grad():  

      for i, frames in enumerate(tqdm(test_loader)):

          frames = frames.float().to(device)             
          
          output = model(frames)

          ### Get most likely predicted phoneme with argmax
          predicted_phonemes = torch.argmax(output,dim=1)

          ### How do you store predicted_phonemes with test_predictions? Hint, look at eval 
          test_predictions.extend(predicted_phonemes.cpu().tolist())

          del frames
          torch.cuda.empty_cache()
          
  return test_predictions

In [ ]:
predictions = test(model, test_loader)

  0%|          | 0/475 [00:00<?, ?it/s]

In [ ]:
### Create CSV file with predictions
with open("./submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(predictions)):
        f.write("{},{}\n".format(i, predictions[i]))

In [ ]:
### Submit to kaggle competition using kaggle API
!kaggle competitions submit -c 11-785-f22-hw1p2 -f ./submission.csv -m "Test Submission"

100% 18.6M/18.6M [00:05<00:00, 3.36MB/s]
Successfully submitted to Frame-Level Speech Recognition

I trained a model, saved it and trained it again by lowering the learning rate (This was because I did not know how to use scheduler properly before). 